In [81]:
!pip install unicodecsv
!pip install gensim
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import unicodecsv                               # csv reader
import re                                       # regular expressions
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier

# To do preprocessing
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
import numpy as np # To compute the average results

from random import shuffle # To shuffle the dataset


# To use feature selection in the Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('large_grammars')
grammar = nltk.data.load('grammars/large_grammars/atis.cfg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package large_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/large_grammars.zip.


In [0]:
# load data from the files and append it
def loadData(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line in reader:
            (Text, Label) = parseReviewImproved(line)
            rawData.append((Text, Label))

def loadData1(path, reviewText=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter=',')
        # print(reader)
        reader.next()
        for line in reader:
            (Text, Label) = parseReviewImproved(line)
            rawData1.append((Text, Label))



# def splitData(percentage):
#     for (Text, Label) in rawData:
#         # print(Text)
#         trainData.append((toFeatureVector(preProcess(Text)),Label))
#     genderf1 = 1

# def splitData1(percentage):
#     for (Text, Label) in rawData1:
#         testData.append((toFeatureVector(preProcess(Text)),Label))
#     genderf2 = 1

def splitData(num):
    for (Text, Label) in rawData:
        # print(Text)
        temp1 = toFeatureVector(preProcess(Text), num)
        # temp2 = wvec_gender_train(temp1)
        temp2 = (temp1,Label)
        trainData.append(temp2)
    # genderf1 = 1

def splitData1(num):
    for (Text, Label) in rawData1:
        temp1 = toFeatureVector(preProcess(Text), num)
        # temp2 = wvec_gender_test(temp1)
        temp2 = (temp1,Label)
        testData.append(temp2)
    # genderf2 = 1

In [0]:
# the output classes
maleLabel = 'male'
femaleLabel = 'female'
labelMap = {'male' : maleLabel, 'female' : femaleLabel}

labelMap1 = {'male' : maleLabel, 'female' : femaleLabel}

# Convert line from input file into an id/text/label tuple plus meta features
def parseReviewImproved(reviewLine):
    # Id    = int(reviewLine[0])
    # Rating = int(reviewLine[2])
    # VerifiedPurchase = reviewLine[3]
    # Category = reviewLine[4]
    Text  = reviewLine[0]
    Label = labelMap[reviewLine[2]]
    return (Text, Label)

In [151]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION

# input: a string of one review
def preProcess(text):
    # should return a list of tokens
    
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = pos_tag(tokens)
    
    # lowercasing
    tokens = [(t.lower(),p) for t,p in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    if False:
        stop = set(stopwords.words('english'))
        tokens = [(t,p) for t,p in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [(lemmatiser.lemmatize(t),p) for t,p in tokens]
    tokens = [(t,p) for t,p in tokens if t] # ensure no empty space
    toks = [t for t,p in tokens]


    # if genderf1 == 0:
    #   gender_whole_train.append(toks)
    # else:
    #   gender_whole_test.append(toks)


    # print(toks)
    # print(grammar)

    # rd_parser = nltk.RecursiveDescentParser(grammar)
    # for tree in rd_parser.parse(['hello','how','are','you']):
    #     print(tree)

    # model = Word2Vec([toks], min_count=1)
    # print(toks)
    # print(model)
    # print(model['hello'])
    # print(model.most_similar('hello'))
    # print(model.wv.most_similar('hello'))
    
    return tokens

print(preProcess("hello this is the, ehh... presumably, a crying situations!"))

[('hello', 'NN'), ('this', 'DT'), ('is', 'VBZ'), ('the', 'DT'), ('ehh', 'NN'), ('presumably', 'RB'), ('a', 'DT'), (u'cry', 'NN'), (u'situation', 'NNS')]


In [0]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION PART 2 (WORD2VEC)
def gen_train_lib(num):
    if num == 1:
        for (Text, Label) in rawData:
            temp1 = gen_helper(Text,num)
        # genderf1 = 1

    elif num == 2:
        for (Text, Label) in rawData1:
            temp1 = gen_helper(Text, num)
        # genderf2 = 1
    
                                   
def gen_helper(text, num):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = pos_tag(tokens)
    
    # lowercasing
    tokens = [(t.lower(),p) for t,p in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    if False:
        stop = set(stopwords.words('english'))
        tokens = [(t,p) for t,p in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [(lemmatiser.lemmatize(t),p) for t,p in tokens]
    tokens = [(t,p) for t,p in tokens if t] # ensure no empty space
    toks = [t for t,p in tokens]

    if num == 1:
      gender_whole_train.append(toks)
    else:
      gender_whole_test.append(toks)

def gen_train():
    gender_model_test = Word2Vec(gender_whole_test, min_count=1)
def gen_test():    
    gender_model_train = Word2Vec(gender_whole_train, min_count=1)


In [0]:
# QUESTION 2
featureDict = {} # the global feature dictionary

def toFeatureVector(tokens, num):
    # return a dictionary 'featureVect' where the keys are the tokens in 'words' and the values are the number of occurrences of the tokens
    # start by using binary values only:
#     baseDict = {}
    featureVec = {}
    prev_tag = {}

    for w, pos in tokens:
        try:
            sum = 0
            featureVec[w] += 1.0/len(tokens)
            # USING POS TAGS. COMMENT OUT THE BELOW SECTION TO OBSERVE SCORE WITHOUT POS TAGS
            if w in prev_tag:
              if pos != prev_tag[w]:
                featureVec[w] -= 1.0/len(tokens)
              else:
                featureVec[w] += 1.0/len(tokens)
            prev_tag[w] = pos
            ########################

            ## USING WORD2VEC ###############
            
            temp_word = w
            if num == 1:
                ret_list = model1.wv.most_similar(temp_word)
            elif num == 2:
                ret_list = model2.wv.most_similar(temp_word)
            length = len(ret_list)
            for x,y in ret_list:
                sum = sum + y
            avg = sum / length
            featureVec[w] += avg

            ##################################




        except KeyError:
            featureVec[w] = 1.0/len(tokens)
        try:
            featureDict[w] += 1.0/len(tokens)
        except KeyError:
            featureDict[w] = 1.0/len(tokens)
    
    # # just get bigram binary presence or not
    # for i in range(1, len(tokens)):
    #     bigram = tokens[0][i-1] + " " + tokens[0][i]
    #     try:
    #         featureVec[bigram] = 1 #+= 1.0/len(tokens)
    #     except KeyError:

    #         featureVec[bigram] = 1 #= 1.0/len(tokens)
    #     try:
    #         featureDict[bigram] += 1.0
    #     except KeyError:
    #         featureDict[bigram] = 1.0
    # print(featureVec)
    return featureVec

In [0]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print "Training Classifier..."
    # pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=948)),('svc', LinearSVC(loss = 'hinge'))])
    # pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=1555)),('svc', LinearSVC(loss = 'squared_hinge'))])
    pipeline =  Pipeline([('tfidf', TfidfTransformer()),('chi2', SelectKBest(chi2, k=1555)),('svc', LinearSVC(loss = 'hinge'))])

    # print("INSIDE")
    return SklearnClassifier(pipeline).train(trainData)

In [0]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(text, classifier):
    return classifier.classify(toFeatureVector(preProcess(text)))

In [0]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    results = []
    foldSize = len(dataset)/folds
    
    for i in range(0,len(dataset),foldSize):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        print "Fold start on items %d - %d" % (i, i+foldSize)
        myTestData = dataset[i:i+foldSize]
        myTrainData = dataset[:i] + dataset[i+foldSize:]
        # print("HERE")
        classifier = trainClassifier(myTrainData)
        # print("HERE1")
        y_true = map(lambda x: x[1], myTestData)
        # print("HERE2")
        y_pred = predictLabels(myTestData, classifier)
        # print("HERE3")
        results.append(precision_recall_fscore_support(y_true, y_pred, average='weighted'))
        
    avgResults = map(np.mean,zip(*results)[:3])
    return avgResults

In [181]:
# MAIN

# loading reviews
rawData = [] # the filtered data from the dataset file for the training Data
trainData = [] # the training data
testData = [] # the test data
rawData1 = [] # the filtered data from the dataset file for the testing Data
gender_whole_train = []
gender_whole_test = []
genderf1 = 0
genderf2 = 0



# references to the data files
trainPath = '/training.csv'


testPath = '/test.csv'

# do the actual stuff
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))
print "Preparing the dataset..."
loadData(trainPath)
loadData1(testPath)


gen_train_lib(1)
gen_train_lib(2)
# print(gender_whole_train)
# print(gender_whole_test)

# stri = [[u'it', u's', u'no', u'problem', u'honestly', u'go', u'on', u'go', u'and', u'open', u'the', u'launderette', u'leave', u'it', u'with', u'me']]
# model = Word2Vec(stri, min_count=1)
model1 = Word2Vec(gender_whole_train, min_count=1)
model2 = Word2Vec(gender_whole_test, min_count=1)
# print("WORD2VEC TEST")
# print(model)
# print(model.wv.most_similar('go'))

# whole1 = [i[0] for i in rawData]
# print(whole1)

# print "Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData))
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))
print "Preparing training and test data..."
splitData(1)
splitData1(2)
# splitData(0.8)
# print "Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData))
print "Now %d rawData, %d rawData1, %d trainData, %d testData" % (len(rawData), len(rawData1), len(trainData), len(testData))


# print(trainData)

Now 0 rawData, 0 rawData1, 0 trainData, 0 testData
Preparing the dataset...
WORD2VEC TEST
Now 10112 rawData, 1123 rawData1, 0 trainData, 0 testData
Preparing training and test data...
Now 10112 rawData, 1123 rawData1, 10112 trainData, 1123 testData


In [182]:
cv_results = crossValidate(trainData, 10)
print cv_results

Fold start on items 0 - 1011
Training Classifier...
Fold start on items 1011 - 2022
Training Classifier...
Fold start on items 2022 - 3033
Training Classifier...
Fold start on items 3033 - 4044
Training Classifier...
Fold start on items 4044 - 5055
Training Classifier...
Fold start on items 5055 - 6066
Training Classifier...
Fold start on items 6066 - 7077
Training Classifier...
Fold start on items 7077 - 8088
Training Classifier...
Fold start on items 8088 - 9099
Training Classifier...
Fold start on items 9099 - 10110
Training Classifier...
Fold start on items 10110 - 11121
Training Classifier...
[0.6005147944087184, 0.5988670083625572, 0.5983256044117549]


In [183]:
classifier = trainClassifier(trainData)
testTrue = map(lambda t: t[1], testData)
testPred = predictLabels(testData, classifier)
finalScores = precision_recall_fscore_support(testTrue, testPred, average='weighted')
finalScoresraw = precision_recall_fscore_support(testTrue, testPred)

print "Done training!"
print("RAW RESULTS = ", finalScoresraw)
print "Precision: %f\nRecall: %f\nF Score:%f" % finalScores[:3]

Training Classifier...
Done training!
('RAW RESULTS = ', (array([0.54620462, 0.62475822]), array([0.63047619, 0.54013378]), array([0.58532272, 0.5793722 ]), array([525, 598])))
Precision: 0.588035
Recall: 0.582369
F Score:0.582154
